# Lab 3: Intro to Distributed Training (Data Parallel)
---

PyTorch 데이터 병렬화에 익숙하신 분들은 이 노트북 섹션을 건더뛰고 다음 섹션으로 진행하세요!


<br>

## 1. PyTorch Training Script for Single GPU
---
앞 모듈의 주피터 노트북 코드를 단일 파이썬 스크립트로 작성한 결과입니다. 

In [ ]:
!pygmentize scripts/train_single.py

In [ ]:
%%time

TRAIN_SINGLE_GPU_CMD = f"""cd scripts && python train_single.py --num_epochs 1 \
    --train_batch_size 32 \
    --eval_batch_size 64 \
    --use_fp16 True
"""

print(f'Running command: \n{TRAIN_SINGLE_GPU_CMD}')
! {TRAIN_SINGLE_GPU_CMD}

<br>

## 2. PyTorch Distributed Data Parallel Tutorial
---

분산 훈련을 위해 상기 스크립트에서 몇 줄의 변환이 필요합니다. 차근차근 알아보도룍 하죠.

#### 기본 용어

- **rank**: 글로벌 프로세스 id (각 GPU는 단일 프로세스에 매칭됩니다)
- **local_rank**: 해당 노드에서의 프로세스 id (a unique local ID for processes running in a single node)
- **node_size**: 독립된 노드의 개수
- **num_gpu**: 각 노드에서 사용할 GPU 개수
- **world_size**: 총 글로벌 프로세스 개수 (node_size * num_gpu)


### 2.1. Setup the process group

각 프로세스당 GPU를 할당하기 위한 초기화를 수행합니다. 이를 통해 여러 노드에 있는 여러 프로세스가 동기화되고 통신합니다.

```python
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

def setup():
    dist.init_process_group(backend="nccl")
    
    if 'WORLD_SIZE' in os.environ:
        # Environment variables set by torch.distributed.launch or torchrun
        world_size = int(os.environ['WORLD_SIZE'])
        rank = int(os.environ['RANK'])
        local_rank = int(os.environ['LOCAL_RANK'])
    elif 'OMPI_COMM_WORLD_SIZE' in os.environ:
        # Environment variables set by mpirun 
        world_size = int(os.environ['OMPI_COMM_WORLD_SIZE'])
        rank = int(os.environ['OMPI_COMM_WORLD_RANK'])
        local_rank = int(os.environ['OMPI_COMM_WORLD_LOCAL_RANK'])
    else:
        sys.exit("Can't find the evironment variables for local rank")
```        

### 2.2. Split the DataLoader to each process

`DataLoader`를 DistributedSampler로 각 프로세스로 분배하고, 각 프로세스당 미니배치가 겹치지 않게 합니다. `DistributedSampler`를 사용하면 전체 데이터를 GPU의 개수로 나눈 부분 데이터셋에서만 데이터를 샘플링합니다. 이 때 주의할 점은 `DistributedSampler`를 사용하면 `DataLoader` 호출 시 `shuffle=False`로 설정해야 합니다!

#### [As-is] Single GPU

```python
train_loader = DataLoader(
    dataset=train_dataset, batch_size=args.train_batch_size, 
    num_workers=4, shuffle=True
)    
eval_loader = DataLoader(
    dataset=eval_dataset, batch_size=args.eval_batch_size, 
    num_workers=4, shuffle=False
)
```

#### [To-be] Distributed Training

```python
train_sampler = DistributedSampler(train_dataset, num_replicas=args.world_size, rank=args.rank)
eval_sampler = DistributedSampler(eval_dataset, num_replicas=args.world_size, rank=args.rank)

train_loader = DataLoader(
    dataset=train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, 
    num_workers=4*torch.cuda.device_count(), shuffle=False
)    
eval_loader = DataLoader(
    dataset=eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, 
    num_workers=4*torch.cuda.device_count(), shuffle=False
)
```

### 2.3. Wrap model to DDP

기존 모델을 DDP()로 래핑합니다. 이는 한 줄의 코드로 간단히 수행 가능합니다.

#### [As-is] Single GPU

```python
model = BertForSequenceClassification.from_pretrained(args.model_id, num_labels=2).to(device)

```

#### [To-be] Distributed Training
 
```python
model = BertForSequenceClassification.from_pretrained(args.model_id, num_labels=2).to(device)
model = DDP(model, device_ids=[device])
```

### 2.4.  set_epoch()

데이터 병렬화 과정에서 각 에폭 시작 시 `set_epoch()` 메서드를 호출해야 미니배치 셔플링이 제대로 작동합니다. 이를 수행하지 않으면 항상 동일한 셔플링이 사용되어 훈련 효과가 떨어집니다.

#### [As-is] Single GPU

```python
for epoch in range(1, args.num_epochs+1):
    ...
```

#### [To-be] Distributed Training
 
```python
for epoch in range(1, args.num_epochs+1):
    train_sampler.set_epoch(epoch)
    eval_sampler.set_epoch(epoch)
    ...
```


### 2.5. Destroy Process group
분산 훈련을 완료하였으면 프로세스 그룹을 종료합니다.

#### [As-is] Single GPU

```python
...
main(args) # main logic
```  

#### [To-be] Distributed Training

```python
...
main(args) # main logic
dist.destroy_process_group()
```   

### (Optional) 2.6. AMP (Automatic Mixed Precision)

모델이 거대해지면서 한정된 GPU 메모리에 모델을 모두 올리기 어렵고 미니배치 크기 또한 키울 수 없는 상황들이 종종 있습니다. 이를 위해 Gradient Checkpointing, 지식 증류, 모델 병렬화 등의 기법을 사용할 수도 있지만, 디폴트로 사용되는 32비트 연산(FP32) 대신 16비트 연산을 사용(FP16)해서 메모리를 절약하고 훈련 속도를 높일 수도 있습니다. PyTorch 구 버전에서는 이를 사용하기 위해서는 별도로 NVIDI의 Apex 라이브러리(https://github.com/NVIDIA/apex)를 설치해야 했지만, PyTorch 1.5.0 버전부터는 AMP 모듈이 기본으로 추가되어 있기에 몇 줄의 코드만으로 쉽게 FP16을 적용할 수 있습니다.


#### Autocasting and Gradient Scaling
특정 연산에 대한 forward 패스가 FP16 입력이 있는 경우, 해당 연산에 대한 backward pass는 FP16 그래디언트(gradient)를 생성하는데, 이 때 크기가 작은 그래디언크 값은 FP16으로 전부 표현할 수 없기에 0으로 세팅되는 언더플로우 현상이 발생합니다. 

![amp1](imgs/amp1.png)
(출처: https://arxiv.org/pdf/1710.03740.pdf)

이 때 loss에 scale factor를 곱하여 scaling된 손실에 backward pass를 호출하면 그래디언트의 크기가 매우 커지므로 FP16이 표현할 수 있는 범위에 들어옵니다. 이를 psuedo 코드로 표현하면 다음과 같습니다.
``` 
scaled_loss = loss * scale_factor
``` 
하지만 backward pass 호출 후 기존 weight를 업데이트할 때는 원래의 스케일로 unscaling을 수행해야겠죠? 이를 몇 줄의 코드로 간단히 적용할 수 있습니다.

좀 더 구체적인 AMP 최적화 옵션은 아래 내용을 참조하세요.
![amp2](imgs/amp2.png)

- **copt_level**c
    - O0: FP32 training
    - O1: [Default] TensorCore을 이용한 FP32 / FP16 혼합 연산으로 TensorCore에 적합한 연산(ops)들은 FP16으로 캐스팅하고 정확한 계산이 필요한 연산들은 FP32를 유지
    - O2: Almost FP16 (BatchNorm weight를 제외한 Model weight가 FP16으로 캐스팅)
    - O3: FP16 training
- **cast_model_type**: 모델 파라메터를 어떤 타입으로 변환할 것인지 여부
- **patch_torch_functions**: 함수를 TensorCore용으로 변환할지 여부
- **keep_batchnorm_fp32**: BatchNorm 연산을 FP32로 유지할지 여부
- **master_weights**: 연산 시의 weight를 FP32로 할지 여부
- **loss_scale**: Gradient Scaling 관련 파라메터

#### [As-is] FP32

```python
def train_model(args, device, model, train_loader, eval_loader, optimizer, lr_scheduler, epoch, pbar):
    model.train()

    for batch_idx, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        
        outputs = model(**batch)
        loss = outputs.loss          
        loss.backward()
        optimizer.step()
        
        lr_scheduler.step()
```        

#### [As-is] AMP Enabled

```python
from torch.cuda.amp import autocast, GradScaler

def train_model(args, device, model, train_loader, eval_loader, optimizer, lr_scheduler, epoch, pbar):
    model.train()

    # AMP (Create gradient scaler)
    scaler = GradScaler(init_scale=16384)
    
    for batch_idx, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast(enabled=args.use_fp16):
            outputs = model(**batch)
            loss = outputs.loss
                
        if args.use_fp16:
            # Backpropagation w/ gradient scaling
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:                
            loss.backward()
            optimizer.step()
        
        lr_scheduler.step()
```        


<br>

## 3. PyTorch Training Script for Distributed Training
---
위 지침대로 코드를 변경한 최종 스크립트입니다.

In [ ]:
!pygmentize scripts/train_pytorchddp.py

`torchrun`으로 분산 훈련 스크립트를 실행합니다. `mpirun`, `torch.multiprocessing.spawn()`, `torch.distributed.launch()` 등의 다양한 방법으로 실행하지만, PyTorch 최신 버전은 `torchrun`으로 분산 훈련을 수행하는 것을 권장합니다.

In [ ]:
%%time

import torch
n_gpus = torch.cuda.device_count()

TRAIN_DDP_CMD = f"""cd scripts && torchrun --nnodes=1 --nproc_per_node={n_gpus} train_pytorchddp.py \
    --train_batch_size 32 \
    --eval_batch_size 64 \
    --use_fp16 True
"""

print(f'Running command: \n{TRAIN_DDP_CMD}')
! {TRAIN_DDP_CMD}


<br>

## 4. SagageMaker Distributed Data Parallel
---
기존 PyTorch DDP 코드에서 거의 변경할 부분이 없습니다. SageMaker 데이터 병렬화 라이브러리를 임포트하고 프로세스 그룹 초기화 시 백엔드를 nccl에서 smddp로 변경하면 됩니다. 다만, 기존 개발 환경에서처럼 곧바로 훈련을 수행할 수 없기에 최소한 로컬 모드를 사용해야 합니다.

#### [As-is] PyTorch DDP

```python
...
dist.init_process_group(backend="nccl")
```  

#### [To-be] SageMaker DDP

```python
...
import smdistributed.dataparallel.torch.torch_smddp
...
dist.init_process_group(backend="smddp")
```   